<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/02-Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai==1.6.0 cohere==4.39 tiktoken==0.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 978.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 963.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

In [116]:
embedding_file = "./index_with_embedding.json"

# Load Dataset

### Download Paul Graham Essay (JSON)

In [6]:
!wget https://raw.githubusercontent.com/run-llama/llama_index/main/examples/paul_graham_essay/index.json

--2023-12-20 16:11:36--  https://raw.githubusercontent.com/run-llama/llama_index/main/examples/paul_graham_essay/index.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176588 (172K) [text/plain]
Saving to: ‘index.json’

index.json          100%[===================>] 172.45K  --.-KB/s    in 0.02s   

2023-12-20 16:11:36 (7.10 MB/s) - ‘index.json’ saved [176588/176588]



### Read JSON

In [69]:
import json

with open('./index.json', 'r') as file:
    data = json.load(file)

In [70]:
data = data['index_struct']['all_nodes']

In [71]:
len( data )

63

In [103]:
import pandas as pd

df = pd.DataFrame(data).transpose()

In [104]:
df.keys()

Index(['text', 'doc_id', 'index', 'child_indices', 'embedding', 'ref_doc_id'], dtype='object')

### Apply Embedding

In [88]:
from openai import OpenAI

client = OpenAI()

def get_embedding(text):
  try:
    # Remove newlines
    text = text.replace("\n", " ")
    res = client.embeddings.create(input = [text], model="text-embedding-ada-002")

    return res.data[0].embedding

  except:
        return None

In [118]:
not bool( embedding_file )

False

In [121]:
from tqdm.notebook import tqdm

if not embedding_file:
  print("Generating embeddings...")
  for index, row in tqdm( df.iterrows() ):
    row['embedding'] = get_embedding( row['text'] )

  df.to_json("./index_with_embedding.json")

else:
  print("Loaded the embedding file.")
  with open('./index_with_embedding.json', 'r') as file:
    data = json.load(file)
  df = pd.DataFrame(data)

Loaded the embedding file.


# User Question

In [293]:
QUESTION = "How much budget Paul Graham had to spend each day when he was in Florence?"

In [294]:
QUESTION_emb = get_embedding( QUESTION )

In [295]:
len( QUESTION_emb )

1536

# Calculate Cosine Similarities

In [296]:
BAD_SOURCE_emb = get_embedding( "The sky is blue." )
GOOD_SOURCE_emb = get_embedding( "Paul Graham had budget of $1 per day in Florence." )

In [297]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity([QUESTION_emb], [BAD_SOURCE_emb]) )
print("> Good Response Score:", cosine_similarity([QUESTION_emb], [GOOD_SOURCE_emb]) )

> Bad Response Score: [[0.71542785]]
> Good Response Score: [[0.94895731]]


In [298]:
cosine_similarities = cosine_similarity( [QUESTION_emb], df['embedding'].tolist() )

In [299]:
cosine_similarities

array([[0.74283738, 0.72700095, 0.75178514, 0.72451674, 0.71790563,
        0.71921944, 0.75048449, 0.74530984, 0.75329709, 0.76653264,
        0.76974562, 0.72420627, 0.73249944, 0.76877011, 0.79248341,
        0.73697747, 0.71247212, 0.78187566, 0.76031472, 0.73767647,
        0.72234783, 0.74282612, 0.75314902, 0.74521852, 0.75741529,
        0.7501051 , 0.75077242, 0.76393937, 0.76099337, 0.76104132,
        0.74024353, 0.71837665, 0.76223564, 0.71520494, 0.72417841,
        0.76740974, 0.74417228, 0.7428031 , 0.74380591, 0.76662179,
        0.76371986, 0.74027053, 0.72705364, 0.76053316, 0.72927648,
        0.72150454, 0.7451773 , 0.69488923, 0.73433034, 0.77833212,
        0.74265443, 0.76117176, 0.7494365 , 0.71624952, 0.72440837,
        0.73051577, 0.72781133, 0.74202563, 0.75499305, 0.7157058 ,
        0.72145234, 0.76054956, 0.71896691]])

In [300]:
import numpy as np

# Sort the scores
highest_index = np.argmax( cosine_similarities )

In [301]:
number_of_chunks_to_retrieve = 3

indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]

In [302]:
indices

array([14, 17, 49])

In [303]:
# Get the highest scored pieces of text
for idx, item in enumerate( df.text[indices] ):
  print(f"> Chunk {idx+1}")
  print(item)
  print("----")

> Chunk 1
of thinking, but at the time it caused a lot of friction. Toward the end of the year I spent much of my time surreptitiously working on On Lisp, which I had by this time gotten a contract to publish.

The good part was that I got paid huge amounts of money, especially by art student standards. In Florence, after paying my part of the rent, my budget for everything else had been $7 a day. Now I was getting paid more than 4 times that every hour, even when I was just sitting in a meeting. By living cheaply I not only managed to save enough to go back to RISD, but also paid off my college loans.

I learned some useful things at Interleaf, though they were mostly about what not to do. I learned that it's better for technology companies to be run by product people than sales people (though sales is a real skill and people who are good at it are really good at it), that it leads to bugs when code is edited by too many people, that cheap office space is no bargain if it's depressing

# Augment the Prompt

In [305]:
try:
    # Formulating the system prompt
    system_prompt = (
      "You are an assistant and expert in answering questions from a chunks of content. "
      "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Combining the system prompt with the user's question
    prompt = (
      "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
      "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
    prompt = prompt.format( "".join( df.text[indices] ), QUESTION )

    # Call the OpenAI API
    response = client.chat.completions.create(
      model='gpt-3.5-turbo-16k',
      temperature=0.0,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
      ]
    )

    # Return the AI's response
    res = response.choices[0].message.content.strip()

except Exception as e:
    print( f"An error occurred: {e}" )

In [306]:
print( res )

Paul Graham had a budget of $7 a day when he was in Florence.


# Without Augmentation

In [307]:
# Formulating the system prompt
system_prompt = (
  "You are an assistant and expert in answering questions."
)

# Combining the system prompt with the user's question
prompt = (
  "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
)
prompt = prompt.format( QUESTION )

# Call the OpenAI API
response = client.chat.completions.create(
  model='gpt-3.5-turbo-16k',
  temperature=.9,
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
  ]
)

# Return the AI's response
res = response.choices[0].message.content.strip()

In [308]:
print( res )

There is no specific information available about Paul Graham's daily budget when he was in Florence.
